In [1]:
! pip install tslearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 6.9 MB/s eta 0:00:00


In [2]:
! pip install scikit-learn-extra

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.1 MB/s eta 0:00:00


In [3]:
! pip install rstl

In [4]:
# Utilities
import os
import csv
import argparse
from time import time
# Keras
import tensorflow as tf

from keras.models import Model
from keras.layers import Dense, Reshape, UpSampling2D, Conv2DTranspose, GlobalAveragePooling1D, Softmax
from keras.losses import kullback_leibler_divergence
import keras.backend as K
# scikit-learn
from sklearn.cluster import AgglomerativeClustering, KMeans
# Dataset helper function
# DTC components
from keras.models import Model
from keras.layers import Input, Conv1D, LeakyReLU, MaxPool1D, LSTM, Bidirectional, TimeDistributed, Dense, Reshape
from keras.layers import UpSampling2D, Conv2DTranspose
import numpy as np
from sklearn import metrics
from scipy.optimize import linear_sum_assignment
from sklearn.preprocessing import label_binarize
from tensorflow.keras.layers import Layer, InputSpec
import keras.backend as K
import numpy as np
from statsmodels.tsa import stattools
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
import tensorflow.keras as keras
from tensorflow.keras.layers import Input, LSTM, Dense, RepeatVector, Attention
from tensorflow.keras import layers
import pandas as pd
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import NearestNeighbors

In [5]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import mean_absolute_error
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
import tensorflow_addons as tfa
from math import pi, ceil

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.models import Model
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from keras.losses import CategoricalCrossentropy

/opt/conda/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids
from sklearn.cluster import SpectralClustering
# %tensorflow_version 1.x
import tensorflow as tf
import keras
import keras.backend as K
from keras import layers
from keras.models import Sequential,Model
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import time
print(tf.__version__)
from keras.layers import MultiHeadAttention
from keras.layers import Dense
import gc
from keras.layers import concatenate
import csv
import math
import warnings
import os

# import xgboost as xgb
warnings.filterwarnings('ignore')
# import GPy, GPyOpt
tfkl = tf.keras.layers
tfk = tf.keras

from rstl import STL
from texttable import Texttable

2.12.0


In [7]:
def autoencoder(latent_dim=16, series_len=None):

    # Input
    x = Input((None, series_len),name='input')
    #masked_seq = layers.Masking(mask_value=0.0)(encoder_inputs)
    # Encoder
    encoded = LSTM(256, return_sequences=True, name="encoded_lstm1")(x)
    encoded = LSTM(128, return_sequences=True, name="encoded_lstm2")(encoded)
    attention = Attention(name="encoded_atten")([encoded, encoded, encoded])  # Apply self-attention to the encoder outputs
    merged = layers.Concatenate(axis=-1)([encoded, attention])        
    encoded = LSTM(64, name="encoded_lstm3")(merged)
    encoded = Dense(latent_dim, name="encoded_dense")(encoded)

    # Decoder
    decoded = RepeatVector(1)(encoded)
    decoded = LSTM(64, return_sequences=True, name="decoded_lstm1")(decoded)
    decoded = LSTM(128, return_sequences=True, name="decoded_lstm2")(decoded)
    decoded = LSTM(256, return_sequences=True, name="decoded_lstm3")(decoded)
    attention = Attention(name="decoded_atten")([decoded, decoded, decoded])  # Apply self-attention to the decoder outputs
    decoded = layers.Concatenate(axis=-1, name="decoded_concat")([decoded, attention])
    decoder_outputs = TimeDistributed(Dense(series_len), name="decoded_timeD")(decoded)
    
    # AE model
    autoencoder = Model(inputs=x, outputs=decoder_outputs, name='AE')

    # Encoder model
    encoder = Model(inputs=x, outputs=encoded, name='encoder')

    # Create input for decoder model
    encoded_input =Input(shape=(latent_dim,))
    # Internal layers in decoder
    decoded = RepeatVector(1)(encoded_input)
    decoded = autoencoder.get_layer('decoded_lstm1')(decoded)
    decoded = autoencoder.get_layer('decoded_lstm2')(decoded)
    decoded = autoencoder.get_layer('decoded_lstm3')(decoded)
    attention = autoencoder.get_layer('decoded_atten')([decoded, decoded, decoded])
    decoded = autoencoder.get_layer('decoded_concat')([decoded, attention])
    decoder_output = autoencoder.get_layer('decoded_timeD')(decoded)
    # Decoder model
    decoder = Model(inputs=encoded_input, outputs=decoder_output, name='decoder')

    return autoencoder, encoder, decoder

In [8]:
def autoencoderNeighbour(latent_dim=16, series_len=None):

    # Input
    x = Input((None, series_len), name='input')
    #masked_seq = layers.Masking(mask_value=0.0)(encoder_inputs)
    # Encoder
    encoded = LSTM(256, return_sequences=True, name='lstm_input_256')(x)
    encoded = LSTM(128, return_sequences=True, name='lstm_input_128')(encoded)
    attention = Attention(name='attention_input')([encoded, encoded, encoded])  # Apply self-attention to the encoder outputs
    merged = layers.Concatenate(axis=-1)([encoded, attention])        
    encoded = LSTM(64, name='lstm_input_64')(merged)
    encoded = Dense(latent_dim, name='dense_input')(encoded)
    
    
    
    
    x_neighbour1 = Input((None, series_len), name='Neighbour1_input')
    x_neighbour2 = Input((None, series_len), name='Neighbour2_input')
    
    encoded_neighbour1 = LSTM(256, return_sequences=True, name='lstm_Neighbour1_256')(x_neighbour1)
    encoded_neighbour2 = LSTM(256, return_sequences=True, name='lstm_Neighbour2_256')(x_neighbour2)
    encoded_neighbour1 = LSTM(128, return_sequences=True, name='lstm_Neighbour1_128')(encoded_neighbour1)
    encoded_neighbour2 = LSTM(128, return_sequences=True, name='lstm_Neighbour2_128')(encoded_neighbour2)
    attention_neighbour1 = Attention(name='attention_Neighbour1')([encoded_neighbour1, encoded_neighbour2, encoded_neighbour2])  # Apply self-attention to the encoder outputs
    attention_neighbour2 = Attention(name='attention_Neighbour2')([encoded_neighbour2, encoded_neighbour1, encoded_neighbour1])  # Apply self-attention to the encoder outputs
    merged_neighbour1 = layers.Concatenate(axis=-1)([encoded_neighbour1, attention_neighbour1])        
    merged_neighbour2 = layers.Concatenate(axis=-1)([encoded_neighbour2, attention_neighbour2])        
    encoded_neighbour1 = LSTM(64, name='lstm_Neighbour1_64')(merged_neighbour1)
    encoded_neighbour2 = LSTM(64, name='lstm_Neighbour2_64')(merged_neighbour2)
    encoded_neighbour1 = Dense(latent_dim, name='dense_Neighbour1')(encoded_neighbour1)
    encoded_neighbour2 = Dense(latent_dim, name='dense_Neighbour2')(encoded_neighbour2)

    out_encoder_Neighbor = layers.Maximum()([encoded_neighbour1, encoded_neighbour2]) 
    # add Final encoder
    encoded = layers.add([encoded ,out_encoder_Neighbor],name='add_inputs')
    
    
    # Decoder
    decoded = RepeatVector(1)(encoded)
    decoded = LSTM(64, return_sequences=True, name="decoded_lstm1")(decoded)
    decoded = LSTM(128, return_sequences=True, name="decoded_lstm2")(decoded)
    decoded = LSTM(256, return_sequences=True, name="decoded_lstm3")(decoded)
    attention = Attention(name="decoded_atten")([decoded, decoded, decoded])  # Apply self-attention to the decoder outputs
    decoded = layers.Concatenate(axis=-1, name="decoded_concat")([decoded, attention])
    decoder_outputs = TimeDistributed(Dense(series_len), name="decoded_timeD")(decoded)
    
    # AE model
    autoencoder = Model(inputs=[x,x_neighbour1,x_neighbour2], outputs=decoder_outputs, name='AE')

    # Encoder model
    encoder = Model(inputs=[x,x_neighbour1,x_neighbour2], outputs=encoded, name='encoder')

    # Create input for decoder model
    encoded_input =Input(shape=(latent_dim,))
    # Internal layers in decoder
    decoded = RepeatVector(1)(encoded_input)
    decoded = autoencoder.get_layer('decoded_lstm1')(decoded)
    decoded = autoencoder.get_layer('decoded_lstm2')(decoded)
    decoded = autoencoder.get_layer('decoded_lstm3')(decoded)
    attention = autoencoder.get_layer('decoded_atten')([decoded, decoded, decoded])
    decoded = autoencoder.get_layer('decoded_concat')([decoded, attention])
    decoder_output = autoencoder.get_layer('decoded_timeD')(decoded)
    # Decoder model
    decoder = Model(inputs=encoded_input, outputs=decoder_output, name='decoder')

    return autoencoder, encoder, decoder

In [9]:
def eucl(x, y):
    """
    Euclidean distance between two multivariate time series given as arrays of shape (timesteps, dim)
    """
    d = np.sqrt(np.sum(np.square(x - y), axis=0))
    return np.sum(d)


def cid(x, y):
    """
    Complexity-Invariant Distance (CID) between two multivariate time series given as arrays of shape (timesteps, dim)
    Reference: Batista, Wang & Keogh (2011). A Complexity-Invariant Distance Measure for Time Series. https://doi.org/10.1137/1.9781611972818.60
    """
    assert(len(x.shape) == 2 and x.shape == y.shape)  # time series must have same length and dimensionality
    ce_x = np.sqrt(np.sum(np.square(np.diff(x, axis=0)), axis=0) + 1e-9)
    ce_y = np.sqrt(np.sum(np.square(np.diff(y, axis=0)), axis=0) + 1e-9)
    d = np.sqrt(np.sum(np.square(x - y), axis=0)) * np.divide(np.maximum(ce_x, ce_y), np.minimum(ce_x, ce_y))
    return np.sum(d)


def cor(x, y):
    """
    Correlation-based distance (COR) between two multivariate time series given as arrays of shape (timesteps, dim)
    """
    scaler = TimeSeriesScalerMeanVariance()
    x_norm = scaler.fit_transform(x)
    y_norm = scaler.fit_transform(y)
    pcc = np.mean(x_norm * y_norm)  # Pearson correlation coefficients
    d = np.sqrt(2.0 * (1.0 - pcc + 1e-9))  # correlation-based similarities
    return np.sum(d)


def acf(x, y):
    """
    Autocorrelation-based distance (ACF) between two multivariate time series given as arrays of shape (timesteps, dim)
    Computes a linearly weighted euclidean distance between the autocorrelation coefficients of the input time series.
    Reference: Galeano & Pena (2000). Multivariate Analysis in Vector Time Series.
    """
    assert (len(x.shape) == 2 and x.shape == y.shape)  # time series must have same length and dimensionality
    x_acf = np.apply_along_axis(lambda z: stattools.acf(z, nlags=z.shape[0]), 0, x)
    y_acf = np.apply_along_axis(lambda z: stattools.acf(z, nlags=z.shape[0]), 0, y)
    weights = np.linspace(1.0, 0.0, x.shape[0])
    d = np.sqrt(np.sum(np.expand_dims(weights, axis=1) * np.square(x_acf - y_acf), axis=0))
    return np.sum(d)

In [10]:
def cluster_acc(y_true, y_pred):
    """
    Calculate unsupervised clustering accuracy. Requires scikit-learn installed

    # Arguments
        y_true: true labels, numpy.array with shape `(n_samples,)`
        y_pred: predicted labels, numpy.array with shape `(n_samples,)`

    # Return
        accuracy, in [0,1]
    """
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    row_ind, col_ind = linear_sum_assignment(w.max() - w)
    return w[row_ind, col_ind].sum() * 1.0 / y_pred.size


def cluster_purity(y_true, y_pred):
    """
    Calculate clustering purity

    # Arguments
        y_true: true labels, numpy.array with shape `(n_samples,)`
        y_pred: predicted labels, numpy.array with shape `(n_samples,)`

    # Return
        purity, in [0,1]
    """
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    label_mapping = w.argmax(axis=1)
    y_pred_voted = y_pred.copy()
    for i in range(y_pred.size):
        y_pred_voted[i] = label_mapping[y_pred[i]]
    return metrics.accuracy_score(y_pred_voted, y_true)


def roc_auc(y_true, q_pred, n_classes):
    """
    Calculate area under ROC curve (ROC AUC)
    WARNING: DO NOT USE, MAY CONTAIN ERRORS
    TODO: CHECK IT!

    # Arguments
        y_true: true labels, numpy.array with shape `(n_samples,)`
        q_pred: predicted probabilities, numpy.array with shape `(n_samples,)`

    # Return
        ROC AUC score, in [0,1]
    """
    if n_classes == 2:  # binary ROC AUC
        auc = max(metrics.roc_auc_score(y_true, q_pred[:, 1]), metrics.roc_auc_score(y_true, q_pred[:, 0]))
    else:  # micro-averaged ROC AUC (multiclass)
        fpr, tpr, _ = metrics.roc_curve(label_binarize(y_true, classes=np.unique(y_true)).ravel(), q_pred.ravel())
        auc = metrics.auc(fpr, tpr)
    return auc

In [11]:
class TSClusteringLayer(Layer):
    """
    Clustering layer converts input sample (feature) to soft label, i.e. a vector that represents the probability of the
    sample belonging to each cluster. The probability is calculated with student's t-distribution.

    # Arguments
        n_clusters: number of clusters.
        weights: list of Numpy array with shape `(n_clusters, timesteps, n_features)` witch represents the initial cluster centers.
        alpha: parameter in Student's t-distribution. Default to 1.0.
        dist_metric: distance metric between sequences used in similarity kernel ('eucl', 'cir', 'cor' or 'acf').
    # Input shape
        3D tensor with shape: `(n_samples, timesteps, n_features)`.
    # Output shape
        2D tensor with shape: `(n_samples, n_clusters)`.
    """

    def __init__(self, n_clusters, weights=None, alpha=1.0, dist_metric='eucl', **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(TSClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.dist_metric = dist_metric
        self.initial_weights = weights
        self.clusters = None
        self.built = False

    def build(self, input_shape):
        self.input_spec = InputSpec(dtype=K.floatx(), shape=(None, input_shape[1]))
        self.clusters = self.add_weight(shape=(self.n_clusters, input_shape[1]), initializer='glorot_uniform', name='cluster_centers')
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    '''def call(self, inputs, **kwargs):
        """
        Student t-distribution kernel, probability of assigning encoded sequence i to cluster k.
            q_{ik} = (1 + dist(z_i, m_k)^2)^{-1} / normalization.

        Arguments:
            inputs: encoded input sequences, shape=(n_samples, timesteps, n_features)
        Return:
            q: soft labels for each sample. shape=(n_samples, n_clusters)
        """
        if self.dist_metric == 'eucl':
            distance = K.sum(K.sqrt(K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=1)), axis=-1)
        elif self.dist_metric == 'cid':
            ce_x = K.sqrt(K.sum(K.square(inputs[:, 1:, :] - inputs[:, :-1, :]), axis=1))  # shape (n_samples, n_features)
            ce_w = K.sqrt(K.sum(K.square(self.clusters[:, 1:, :] - self.clusters[:, :-1, :]), axis=1))  # shape (n_clusters, n_features)
            ce = K.maximum(K.expand_dims(ce_x, axis=1), ce_w) / K.minimum(K.expand_dims(ce_x, axis=1), ce_w)  # shape (n_samples, n_clusters, n_features)
            ed = K.sqrt(K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2))  # shape (n_samples, n_clusters, n_features)
            distance = K.sum(ed * ce, axis=-1)  # shape (n_samples, n_clusters)
        elif self.dist_metric == 'cor':
            inputs_norm = (inputs - K.expand_dims(K.mean(inputs, axis=1), axis=1)) / K.expand_dims(K.std(inputs, axis=1), axis=1)  # shape (n_samples, timesteps, n_features)
            clusters_norm = (self.clusters - K.expand_dims(K.mean(self.clusters, axis=1), axis=1)) / K.expand_dims(K.std(self.clusters, axis=1), axis=1)  # shape (n_clusters, timesteps, n_features)
            pcc = K.mean(K.expand_dims(inputs_norm, axis=1) * clusters_norm, axis=2)  # Pearson correlation coefficients
            distance = K.sum(K.sqrt(2.0 * (1.0 - pcc)), axis=-1)  # correlation-based similarities, shape (n_samples, n_clusters)
        elif self.dist_metric == 'acf':
            raise NotImplementedError
        else:
            raise ValueError('Available distances are eucl, cid, cor and acf!')
        q = 1.0 / (1.0 + K.square(distance) / self.alpha)
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
        return q'''
    
    def call(self, inputs, **kwargs):
        """
        Student t-distribution kernel, probability of assigning encoded sequence i to cluster k.
            q_{ik} = (1 + dist(z_i, m_k)^2)^{-1} / normalization.

        Arguments:
            inputs: encoded input sequences, shape=(n_samples, n_features)
        Return:
            q: soft labels for each sample. shape=(n_samples, n_clusters)
        """
        inputs_expanded = K.expand_dims(inputs, axis=1)  # shape=(n_samples, 1, n_features)
        clusters_expanded = K.expand_dims(self.clusters, axis=0)  # shape=(1, n_clusters, n_features)

        if self.dist_metric == 'eucl':
            distance = K.sqrt(K.sum(K.square(inputs_expanded - clusters_expanded), axis=-1))
        elif self.dist_metric == 'cid':
            ce_x = K.sqrt(K.sum(K.square(inputs_expanded - K.expand_dims(inputs[:, :-1], axis=2)), axis=-1))  # shape (n_samples, timesteps)
            ce_w = K.sqrt(K.sum(K.square(clusters_expanded - K.expand_dims(self.clusters[:, :-1], axis=1)), axis=-1))  # shape (n_clusters, timesteps)
            ce = K.maximum(K.expand_dims(ce_x, axis=1), ce_w) / K.minimum(K.expand_dims(ce_x, axis=1), ce_w)  # shape (n_samples, n_clusters, timesteps)
            ed = K.sqrt(K.sum(K.square(inputs_expanded - clusters_expanded), axis=-1))  # shape (n_samples, n_clusters)
            distance = K.sum(ed * ce, axis=-1)  # shape (n_samples, n_clusters)
        elif self.dist_metric == 'cor':
            inputs_norm = (inputs - K.mean(inputs, axis=1, keepdims=True)) / K.std(inputs, axis=1, keepdims=True)  # shape (n_samples, n_features)
            clusters_norm = (self.clusters - K.mean(self.clusters, axis=1, keepdims=True)) / K.std(self.clusters, axis=1, keepdims=True)  # shape (n_clusters, n_features)
            pcc = K.mean(K.expand_dims(inputs_norm, axis=1) * K.expand_dims(clusters_norm, axis=0), axis=-1)  # Pearson correlation coefficients
            distance = K.sqrt(2.0 * (1.0 - pcc))  # correlation-based similarities, shape (n_samples, n_clusters)
        elif self.dist_metric == 'acf':
            raise NotImplementedError
        else:
            raise ValueError('Available distances are eucl, cid, cor and acf!')

        q = 1.0 / (1.0 + K.square(distance) / self.alpha)
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
        return q


    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 3
        return input_shape[0], self.n_clusters

    def get_config(self):
        config = {'n_clusters': self.n_clusters, 'dist_metric': self.dist_metric}
        base_config = super(TSClusteringLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [12]:
m3 = pd.read_csv("/kaggle/input/m3monthdataset/M3Month.csv")
m3.head()

,Series,N,NF,Category,Starting Year,Starting Month,1,2,3,4,...,135,136,137,138,139,140,141,142,143,144
0,N1402,68,18,MICRO,1990,1,2640.0,2640.0,2160.0,4200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,N1403,68,18,MICRO,1990,1,1680.0,1920.0,120.0,1080.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,N1404,68,18,MICRO,1990,1,1140.0,720.0,4860.0,1200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,N1405,68,18,MICRO,1990,1,180.0,940.0,2040.0,800.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,N1406,68,18,MICRO,1990,1,2000.0,1550.0,4450.0,3050.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
class_dataframes = {}  # A dictionary to store DataFrames for each class

# Iterate over unique class labels in the 'category' column
for class_label in m3['Category'].unique():
    print(class_label)
    # Filter the original DataFrame for rows with the current class label
    class_df = m3[m3['Category'] == class_label]
    class_label = class_label.replace(" ", "")
    # Store the class-specific DataFrame in the dictionary with the class label as the key
    class_dataframes[class_label] = class_df

MICRO       
INDUSTRY    
MACRO       
FINANCE     
DEMOGRAPHIC 
OTHER       


In [14]:
def preprocessing(data):
    ts_train = []

    for i in range(data.shape[0]):
        temp=np.array(list(data.iloc[i][6:].dropna()))
        temp = temp[:-18]
        temp=temp.reshape(1,len(temp),1)
        temp2 = TimeSeriesScalerMeanVariance().fit_transform(temp)
        ts_train.append(temp2.reshape(-1,1))

    return ts_train 

In [15]:
class DTC:
    def __init__(self, n_clusters, input_dim, timesteps,max_dim=None,dataset_name=None,
                 alpha=1.0, dist_metric='eucl', cluster_init='kmeans', heatmap=False):
        self.n_clusters = 2
        self.input_dim = None
        self.max_dim=max_dim
        self.dataset_name=dataset_name
        self.timesteps = 791
        self.latent_shape = 16
        self.alpha = alpha
        self.dist_metric = dist_metric
        self.cluster_init = cluster_init
        self.heatmap = heatmap
        self.pretrained = False
        self.alpha2 = 0.8
        self.learning_rate = None
        self.optimizer = keras.optimizers.Adam()
        self.model = self.autoencoder = self.encoder = self.decoder = self.predmodel =  None
        self.autoencoderNeighbour = self.encoderNeighbour = self.decoderNeighbour =  None
        self.FeatureWeight = None
        self.a = 1.0
        self.b = 1.0
        self.initial_weights = None
        self.a_b = [1.93, 0.79]
        self.KNN = None
        self.b1 = -0.9
        self.b2 = self.b1/2

        
    def initialize(self):
        """
        Create DTC model
        """
        # Create AE models
        self.FeatureWeight = np.ones((self.n_clusters, self.latent_shape),dtype='float32')/self.latent_shape
        self.autoencoder, self.encoder, self.decoder = autoencoder(series_len=self.max_dim)
        self.autoencoderNeighbour , self.encoderNeighbour , self.decoderNeighbour = autoencoderNeighbour(series_len=self.max_dim)
        clustering_layer = TSClusteringLayer(self.n_clusters,
                                             alpha=self.alpha,
                                             dist_metric=self.dist_metric,
                                             name='TSClustering')(self.encoderNeighbour.output)
        # Create DTC model
        self.model = Model(inputs=[self.autoencoderNeighbour.input],
                               outputs=[self.autoencoderNeighbour.output, clustering_layer])
        self.KNN = NearestNeighbors(n_neighbors=3,metric='euclidean')

    @property
    def cluster_centers_(self):
        """
        Returns cluster centers
        """
        return self.model.get_layer(name='TSClustering').get_weights()[0]
    
    def reconstruction_loss(self, data, reconstruction):
        reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.mean_squared_error(data, reconstruction), axis=(0, 1)
                )
            )
        return reconstruction_loss

    @staticmethod
    def weighted_kld(loss_weight):
        """
        Custom KL-divergence loss with a variable weight parameter
        """
        def loss(y_true, y_pred):
            return loss_weight * kullback_leibler_divergence(y_true, y_pred)
        return loss
    
    def loss_kld(self, y_true, y_pred):
        return kullback_leibler_divergence(y_true, y_pred)
    
    
    def loss_kld(self, y_true, y_pred):
        return kullback_leibler_divergence(y_true, y_pred)
    
    def categorical_cross_entropy(self, y_true, y_pred):
        cce = CategoricalCrossentropy()
        return cce(y_true, y_pred)
        
    def on_epoch_end(self, epoch):
        """
        Update heatmap loss weight on epoch end
        """
        if epoch > self.finetune_heatmap_at_epoch:
            K.set_value(self.heatmap_loss_weight, self.final_heatmap_loss_weight)

    def compile(self, gamma, optimizer, initial_heatmap_loss_weight=None, final_heatmap_loss_weight=None):
        """
        Compile DTC model

        # Arguments
            gamma: coefficient of TS clustering loss
            optimizer: optimization algorithm
            initial_heatmap_loss_weight (optional): initial weight of heatmap loss vs clustering loss
            final_heatmap_loss_weight (optional): final weight of heatmap loss vs clustering loss (heatmap finetuning)
        """
        if self.heatmap:
            self.initial_heatmap_loss_weight = initial_heatmap_loss_weight
            self.final_heatmap_loss_weight = final_heatmap_loss_weight
            self.heatmap_loss_weight = K.variable(self.initial_heatmap_loss_weight)
            self.model.compile(loss=['mse', DTC.weighted_kld(1.0 - self.heatmap_loss_weight), DTC.weighted_kld(self.heatmap_loss_weight)],
                               loss_weights=[1.0, gamma, gamma],
                               optimizer=optimizer)
        else:
            self.model.compile(loss=['mse', 'CategoricalCrossentropy'],
                               loss_weights=[1.0, gamma],
                               optimizer=optimizer)

    def load_weights(self, weights_path):
        """
        Load pre-trained weights of DTC model

        # Arguments
            weight_path: path to weights file (.h5)
        """
        self.model.load_weights(weights_path)
        self.pretrained = True

    def load_ae_weights(self, ae_weights_path):
        """
        Load pre-trained weights of AE

        # Arguments
            ae_weight_path: path to weights file (.h5)
        """
        self.autoencoder.load_weights(ae_weights_path)
        self.pretrained = True
        
    def load_ae_neighbour_weights(self):
        input = self.autoencoder.get_layer(name='input').get_weights()
        encoder_0 = self.autoencoder.get_layer(name='encoded_lstm1').get_weights()
        encoder_1 = self.autoencoder.get_layer(name='encoded_lstm2').get_weights()
        encoder_2 = self.autoencoder.get_layer(name='encoded_lstm3').get_weights()
        atten_encode = self.autoencoder.get_layer(name='encoded_atten').get_weights()
        dense_encode = self.autoencoder.get_layer(name="encoded_dense").get_weights()
        decoder_0 = self.autoencoder.get_layer(name='decoded_lstm1').get_weights()
        decoder_1 = self.autoencoder.get_layer(name='decoded_lstm2').get_weights()
        decoder_2 = self.autoencoder.get_layer(name='decoded_lstm3').get_weights()
        decoder_atten = self.autoencoder.get_layer(name='decoded_atten').get_weights()
        decoder_timeD = self.autoencoder.get_layer(name="decoded_timeD").get_weights()

        # Image weight
        self.autoencoderNeighbour.get_layer(name='input').set_weights(input)
        self.autoencoderNeighbour.get_layer(name='lstm_input_256').set_weights(encoder_0)
        self.autoencoderNeighbour.get_layer(name='lstm_input_128').set_weights(encoder_1)
        self.autoencoderNeighbour.get_layer(name='lstm_input_64').set_weights(encoder_2)
        self.autoencoderNeighbour.get_layer(name='attention_input').set_weights(atten_encode)
        self.autoencoderNeighbour.get_layer(name='dense_input').set_weights(dense_encode)
        
        

        
        self.autoencoderNeighbour.get_layer(name='decoded_lstm1').set_weights(decoder_0)
        self.autoencoderNeighbour.get_layer(name='decoded_lstm2').set_weights(decoder_1)
        self.autoencoderNeighbour.get_layer(name='decoded_lstm3').set_weights(decoder_2)
        self.autoencoderNeighbour.get_layer(name='decoded_atten').set_weights(decoder_atten)

        self.autoencoderNeighbour.get_layer(name='lstm_Neighbour1_256').set_weights(encoder_0)
        self.autoencoderNeighbour.get_layer(name='lstm_Neighbour1_128').set_weights(encoder_1)
        self.autoencoderNeighbour.get_layer(name='lstm_Neighbour1_64').set_weights(encoder_2)
        self.autoencoderNeighbour.get_layer(name='attention_Neighbour1').set_weights(decoder_atten)
        
        self.autoencoderNeighbour.get_layer(name='lstm_Neighbour2_256').set_weights(encoder_0)
        self.autoencoderNeighbour.get_layer(name='lstm_Neighbour2_128').set_weights(encoder_1)
        self.autoencoderNeighbour.get_layer(name='lstm_Neighbour2_64').set_weights(encoder_2)
        self.autoencoderNeighbour.get_layer(name='attention_Neighbour2').set_weights(decoder_atten)

        self.autoencoderNeighbour.get_layer(name='Neighbour1_input').set_weights(input)
        self.autoencoderNeighbour.get_layer(name='Neighbour2_input').set_weights(input)

        # Image weight (encoder)
        self.encoderNeighbour.get_layer(name='input').set_weights(input)
        self.encoderNeighbour.get_layer(name='lstm_input_256').set_weights(encoder_0)
        self.encoderNeighbour.get_layer(name='lstm_input_128').set_weights(encoder_1)
        self.encoderNeighbour.get_layer(name='lstm_input_64').set_weights(encoder_2)
        self.encoderNeighbour.get_layer(name='attention_input').set_weights(atten_encode)
        
        self.encoderNeighbour.get_layer(name='lstm_Neighbour1_256').set_weights(encoder_0)
        self.encoderNeighbour.get_layer(name='lstm_Neighbour1_128').set_weights(encoder_1)
        self.encoderNeighbour.get_layer(name='lstm_Neighbour1_64').set_weights(encoder_2)
        self.encoderNeighbour.get_layer(name='attention_Neighbour1').set_weights(decoder_atten)
        
        self.encoderNeighbour.get_layer(name='lstm_Neighbour2_256').set_weights(encoder_0)
        self.encoderNeighbour.get_layer(name='lstm_Neighbour2_128').set_weights(encoder_1)
        self.encoderNeighbour.get_layer(name='lstm_Neighbour2_64').set_weights(encoder_2)
        self.encoderNeighbour.get_layer(name='attention_Neighbour2').set_weights(decoder_atten)

        self.encoderNeighbour.get_layer(name='Neighbour1_input').set_weights(input)
        self.encoderNeighbour.get_layer(name='Neighbour2_input').set_weights(input)

    def dist(self, x1, x2):
        """
        Compute distance between two multivariate time series using chosen distance metric

        # Arguments
            x1: first input (np array)
            x2: second input (np array)
        # Return
            distance
        """
        if self.dist_metric == 'eucl':
            return tsdistances.eucl(x1, x2)
        elif self.dist_metric == 'cid':
            return tsdistances.cid(x1, x2)
        elif self.dist_metric == 'cor':
            return tsdistances.cor(x1, x2)
        elif self.dist_metric == 'acf':
            return tsdistances.acf(x1, x2)
        else:
            raise ValueError('Available distances are eucl, cid, cor and acf!')

    def init_cluster_weights(self, X):
        """
        Initialize with complete-linkage hierarchical clustering or k-means.

        # Arguments
            X: numpy array containing training set or batch
        """
        assert(self.cluster_init in ['hierarchical', 'kmeans'])
        print('Initializing cluster...')

        features = self.encode(X)

        if self.cluster_init == 'hierarchical':
            if self.dist_metric == 'eucl':  # use AgglomerativeClustering off-the-shelf
                hc = AgglomerativeClustering(n_clusters=self.n_clusters,
                                             affinity='euclidean',
                                             linkage='complete').fit(features.reshape(features.shape[0], -1))
            else:  # compute distance matrix using dist
                d = np.zeros((features.shape[0], features.shape[0]))
                for i in range(features.shape[0]):
                    for j in range(i):
                        d[i, j] = d[j, i] = self.dist(features[i], features[j])
                hc = AgglomerativeClustering(n_clusters=self.n_clusters,
                                             affinity='precomputed',
                                             linkage='complete').fit(d)
            # compute centroid
            cluster_centers = np.array([features[hc.labels_ == c].mean(axis=0) for c in range(self.n_clusters)])
        elif self.cluster_init == 'kmeans':
            # fit k-means on flattened features
            km = KMeans(n_clusters=self.n_clusters, n_init=10).fit(features.reshape(features.shape[0], -1))
            cluster_centers = km.cluster_centers_.reshape(self.n_clusters, features.shape[1])

        self.model.get_layer(name='TSClustering').set_weights([cluster_centers])
        print('Done!')

    def encode(self, x):
        """
        Encoding function. Extract latent features from hidden layer

        # Arguments
            x: data point
        # Return
            encoded (latent) data point
        """
        return self.encoder.predict(x)

    def decode(self, x):
        """
        Decoding function. Decodes encoded sequence from latent space.

        # Arguments
            x: encoded (latent) data point
        # Return
            decoded data point
        """
        return self.decoder.predict(x)

    def predict(self, x):
        """
        Predict cluster assignment.

        """
        q = self.model.predict(x, verbose=0)[1]
        return q.argmax(axis=1)
    
    @staticmethod
    def target_distribution(q):  # target distribution p which enhances the discrimination of soft label q
        weight = q ** 2 / q.sum(0)
        return (weight.T / weight.sum(1)).T

    def predict_heatmap(self, x):
        """
        Produces TS clustering heatmap from input sequence.

        # Arguments
            x: data point
        # Return
            heatmap
        """
        return self.heatmap_model.predict(x, verbose=0)

    def pretrain(self, X,
                 optimizer='adam',
                 epochs=10,
                 batch_size=64,
                 save_dir='results/tmp',
                 verbose=1):
        """
        Pre-train the autoencoder using only MSE reconstruction loss
        Saves weights in h5 format.

        # Arguments
            X: training set
            optimizer: optimization algorithm
            epochs: number of pre-training epochs
            batch_size: training batch size
            save_dir: path to existing directory where weights will be saved
        """
        print('Pretraining...')
        self.autoencoder.compile(optimizer=optimizer, loss='mse')

        # Begin pretraining
        #t0 = time()
        self.autoencoder.fit(X, X, batch_size=batch_size, epochs=epochs)
        #print('Pretraining time: ', time() - t0)
        self.autoencoder.save_weights('{}/ae_weights-epoch{}.h5'.format(save_dir, epochs))
        print('Pretrained weights are saved to {}/ae_weights-epoch{}.h5'.format(save_dir, epochs))
        self.pretrained = True
        
    def mutate_clustering(self, clustering, mutation_rate=0.3):
        new_clustering = clustering.copy()
        n_samples = len(clustering)
        for i in range(n_samples):
            if np.random.rand() < mutation_rate:
                new_clustering[i] = 1 - new_clustering[i]
        return new_clustering

    def crossover_clustering(self, clustering1, clustering2, crossover_rate=0.3):
        n_samples = len(clustering1)
        new_clustering = clustering1.copy()
        for i in range(n_samples):
            if np.random.rand() < crossover_rate:
                new_clustering[i] = clustering2[i]
        return new_clustering
    
    def save_metrics(self, metrics, file_path='metrics.csv'):
        """
        Save training metrics to a CSV file.

        # Arguments
            metrics: List of dictionaries containing metrics for each epoch
            file_path: Path to the CSV file where metrics will be saved
        """
        df = pd.DataFrame(metrics)
        df.to_csv(file_path, index=False)
        print(f'Metrics saved to {file_path}')
    
    def fit(self, X_train, y_train=None, X_val=None, y_val=None,
        epochs=10,
        eval_epochs=10,
        save_epochs=10,
        batch_size=64,
        tol=0.001,
        patience=5,
        finetune_heatmap_at_epoch=8,
        save_dir='results/tmp',
        mutation_rate=0.01,
        crossover_rate=0.5,
        num_iterations=5):
        
    
            if not self.pretrained:
                print('Autoencoder was not pre-trained!')


            patience_cnt = 0
            index = 0
            print('Training for {} epochs.\nEvaluating every {} and saving model every {} epochs.'.format(epochs, eval_epochs, save_epochs))
            index_array = np.arange(X_train.shape[0])
            for epoch in range(epochs):
                # Initial clustering
                if epoch == 0:
                    q = self.model.predict([X_train, X_train, X_train])[1]
                    extract = Model(inputs = self.model.input, outputs = self.model.get_layer('add_inputs').output)
                    z = extract.predict([X_train, X_train, X_train])
                else:
                    self.KNN.fit(z)
                    _,Neighbors_list=self.KNN.kneighbors(z)
                    q = self.model.predict([X_train,X_train[Neighbors_list[:,1]], X_train[Neighbors_list[:,2]]])[1]

                p = DTC.target_distribution(q)
                p_pred = p.argmax(axis=1)
                reliableindex, _ = np.unique(np.unique(np.where(np.sort(q,axis=1)[:,-1] - np.sort(q,axis=1)[:,-2]>=self.b2), np.where(np.max(q,axis=1)>=self.b1)),
                           np.unique(np.where(np.sort(p,axis=1)[:,-1] - np.sort(p,axis=1)[:,-2]>=self.b2), np.where(np.max(p,axis=1)>=self.b1)))

                print('Number of reliable samples:', len(q[reliableindex]) )
                print('Number of unreliable samples:', len(q)-len(q[reliableindex]) )
#                 print("my",new_clustering)
                Number_Unreliable = len(q)-len(q[reliableindex])
                Number_reliable = len(q[reliableindex])
                with tf.GradientTape() as tape:
                    idx = index_array[index:reliableindex.shape[0]]
                    self.KNN.fit(z[reliableindex[idx]])
                    _,Neighbors_list=self.KNN.kneighbors(z[reliableindex[idx]])
                    outputs = self.model([X_train[reliableindex[idx]], X_train[Neighbors_list[:,1]], X_train[Neighbors_list[:,2]]], training=True)
                    reconstruction_output, clustering_output = outputs
                    reconstruction_loss = self.reconstruction_loss(X_train[reliableindex[idx]], reconstruction_output)
                    clustering_loss = self.categorical_cross_entropy(p[reliableindex[idx]], clustering_output)
                    total_loss = tf.reduce_sum(reconstruction_loss) + tf.reduce_sum(clustering_loss)
                    total_loss = tf.reduce_sum(total_loss)
                    print("reconstruction_loss: ",tf.reduce_sum(reconstruction_loss))
                    print("clustering_loss: ",tf.reduce_sum(clustering_loss))

                            
                grads = tape.gradient(total_loss, self.model.trainable_variables)
                self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))
                num_elements = len(reliableindex[idx])
                n1 = Neighbors_list[:,1]
                n1 = n1[:num_elements]
                n2 = Neighbors_list[:,2]
                n2 = n2[:num_elements]
                z[idx] = extract.predict([X_train[reliableindex[idx]], X_train[n1], X_train[n2]])
                print("Epoch: {}/{}".format(epoch+1, epochs))
                print("Total Loss: ", total_loss)

In [16]:
dataset_MICRO = preprocessing(class_dataframes['MICRO'])
dataset_INDUSTRY = preprocessing(class_dataframes['INDUSTRY'])
dataset_MACRO = preprocessing(class_dataframes['MACRO'])
dataset_FINANCE = preprocessing(class_dataframes['FINANCE'])
dataset_DEMOGRAPHIC = preprocessing(class_dataframes['DEMOGRAPHIC'])
dataset_OTHER = preprocessing(class_dataframes['OTHER'])

In [17]:
max_seq_length_MICRO = max(len(seq) for seq in dataset_MICRO)
max_seq_length_INDUSTRY = max(len(seq) for seq in dataset_INDUSTRY)
max_seq_length_MACRO = max(len(seq) for seq in dataset_MACRO)
max_seq_length_FINANCE = max(len(seq) for seq in dataset_FINANCE)
max_seq_length_DEMOGRAPHIC = max(len(seq) for seq in dataset_DEMOGRAPHIC)
max_seq_length_OTHER = max(len(seq) for seq in dataset_OTHER)

In [18]:
padded_sequences_MICRO = tf.keras.preprocessing.sequence.pad_sequences(dataset_MICRO, maxlen=max_seq_length_MICRO, padding='post', dtype='float32')
padded_sequences_INDUSTRY = tf.keras.preprocessing.sequence.pad_sequences(dataset_INDUSTRY, maxlen=max_seq_length_INDUSTRY, padding='post', dtype='float32')
padded_sequences_MACRO = tf.keras.preprocessing.sequence.pad_sequences(dataset_MACRO, maxlen=max_seq_length_MACRO, padding='post', dtype='float32')
padded_sequences_FINANCE = tf.keras.preprocessing.sequence.pad_sequences(dataset_FINANCE, maxlen=max_seq_length_FINANCE, padding='post', dtype='float32')
padded_sequences_DEMOGRAPHIC = tf.keras.preprocessing.sequence.pad_sequences(dataset_DEMOGRAPHIC, maxlen=max_seq_length_DEMOGRAPHIC, padding='post', dtype='float32')
padded_sequences_OTHER = tf.keras.preprocessing.sequence.pad_sequences(dataset_OTHER, maxlen=max_seq_length_OTHER, padding='post', dtype='float32')

In [19]:
reshaped_array_MICRO = padded_sequences_MICRO.reshape(padded_sequences_MICRO.shape[0], 1, padded_sequences_MICRO.shape[1])
reshaped_array_INDUSTRY = padded_sequences_INDUSTRY.reshape(padded_sequences_INDUSTRY.shape[0], 1, padded_sequences_INDUSTRY.shape[1])
reshaped_array_MACRO = padded_sequences_MACRO.reshape(padded_sequences_MACRO.shape[0], 1, padded_sequences_MACRO.shape[1])
reshaped_array_FINANCE = padded_sequences_FINANCE.reshape(padded_sequences_FINANCE.shape[0], 1, padded_sequences_FINANCE.shape[1])
reshaped_array_DEMOGRAPHIC = padded_sequences_DEMOGRAPHIC.reshape(padded_sequences_DEMOGRAPHIC.shape[0], 1, padded_sequences_DEMOGRAPHIC.shape[1])
reshaped_array_OTHER = padded_sequences_OTHER.reshape(padded_sequences_OTHER.shape[0], 1, padded_sequences_OTHER.shape[1])

In [20]:
print(reshaped_array_MICRO.shape)
print(reshaped_array_INDUSTRY.shape)
print(reshaped_array_MACRO.shape)
print(reshaped_array_FINANCE.shape)
print(reshaped_array_DEMOGRAPHIC.shape)
print(reshaped_array_OTHER.shape)

(474, 1, 108)
(334, 1, 126)
(312, 1, 126)
(145, 1, 126)
(111, 1, 120)
(52, 1, 102)


In [21]:
# Instantiate model
# dtc = DTC(n_clusters=2,input_dim=reshaped_array_DEMOGRAPHIC.shape[-1], timesteps=reshaped_array_DEMOGRAPHIC.shape[1], max_dim=max_seq_length_DEMOGRAPHIC,dataset_name='m3-demo')

In [22]:
# Instantiate model
# dtc = DTC(n_clusters=2,input_dim=reshaped_array_cif012.shape[-1], timesteps=reshaped_array_cif012.shape[1])

In [23]:
def preprocessing_tourism_hospital(data):
    ts_train = []

    for i in range(data.shape[0]):
        temp=np.array(list(data.iloc[i][:].dropna()))
#         temp = temp[:-18]
        temp=temp.reshape(1,len(temp),1)
        temp2 = TimeSeriesScalerMeanVariance().fit_transform(temp)
        ts_train.append(temp2.reshape(-1,1))

    return ts_train 

In [24]:
dataset_tourism = pd.read_excel("/kaggle/input/newtsdatasets/Tourism-new.xlsx",header=None) # Kaggle - Wikipedia - Horizon 59

dataset_tourism=dataset_tourism.iloc[:,:-8]
dataset_tourism = preprocessing_tourism_hospital(dataset_tourism)
max_seq_length_tourism= max(len(seq) for seq in dataset_tourism)
padded_sequences_tourism = tf.keras.preprocessing.sequence.pad_sequences(dataset_tourism, maxlen=max_seq_length_tourism, padding='post', dtype='float32')
reshaped_array_tourism = padded_sequences_tourism.reshape(padded_sequences_tourism.shape[0], 1, padded_sequences_tourism.shape[1])
print(reshaped_array_tourism.shape)

(427, 1, 122)


In [25]:
dataset_hospital = pd.read_excel("/kaggle/input/newtsdatasets/Hospital_new.xlsx",header=None) # Kaggle - Wikipedia - Horizon 59

dataset_hospital=dataset_hospital.iloc[:,:-12]
dataset_hospital = preprocessing_tourism_hospital(dataset_hospital)
max_seq_length_hospital= max(len(seq) for seq in dataset_hospital)
padded_sequences_hospital = tf.keras.preprocessing.sequence.pad_sequences(dataset_hospital, maxlen=max_seq_length_hospital, padding='post', dtype='float32')
reshaped_array_hospital= padded_sequences_hospital.reshape(padded_sequences_hospital.shape[0], 1, padded_sequences_hospital.shape[1])
print(reshaped_array_hospital.shape)

(767, 1, 72)


In [26]:
dataset_cif = pd.read_excel("/kaggle/input/cifnewdataset/12.xlsx",header=None) # Kaggle - Wikipedia - Horizon 59

dataset_cif=dataset_cif.iloc[:,:-12]
dataset_cif = preprocessing_tourism_hospital(dataset_cif)
max_seq_length_cif= max(len(seq) for seq in dataset_cif)
padded_sequences_cif = tf.keras.preprocessing.sequence.pad_sequences(dataset_cif, maxlen=max_seq_length_cif, padding='post', dtype='float32')
reshaped_array_cif= padded_sequences_cif.reshape(padded_sequences_cif.shape[0], 1, padded_sequences_cif.shape[1])
print(reshaped_array_cif.shape)

(57, 1, 108)


In [27]:
dataset_name='m3-micro'

In [28]:
if dataset_name=='cif-12':
    padded_seq=padded_sequences_cif
    reshaped_array=reshaped_array_cif
    max_dim=max_seq_length_cif
elif dataset_name=='hospital':
    padded_seq=padded_sequences_hospital
    reshaped_array=reshaped_array_hospital
    max_dim=max_seq_length_hospital
elif dataset_name=='tourism':
    padded_seq=padded_sequences_tourism
    reshaped_array=reshaped_array_tourism
    max_dim=max_seq_length_tourism
elif dataset_name=='m3-demo':
    padded_seq=padded_sequences_DEMOGRAPHIC
    reshaped_array=reshaped_array_DEMOGRAPHIC
    max_dim=max_seq_length_DEMOGRAPHIC
elif dataset_name=='m3-finance':
    padded_seq=padded_sequences_FINANCE
    reshaped_array=reshaped_array_FINANCE
    max_dim=max_seq_length_FINANCE
elif dataset_name=='m3-industry':
    padded_seq=padded_sequences_INDUSTRY
    reshaped_array=reshaped_array_INDUSTRY
    max_dim=max_seq_length_INDUSTRY

elif dataset_name=='m3-macro':
    padded_seq=padded_sequences_MACRO
    reshaped_array=reshaped_array_MACRO
    max_dim=max_seq_length_MACRO
elif dataset_name=='m3-micro':
    padded_seq=padded_sequences_MICRO
    reshaped_array=reshaped_array_MICRO
    max_dim=max_seq_length_MICRO
else:
    padded_seq=padded_sequences_OTHER
    reshaped_array=reshaped_array_OTHER
    max_dim=max_seq_length_OTHER

In [29]:
dtc = DTC(n_clusters=2,input_dim=reshaped_array.shape[-1], timesteps=padded_seq.shape[1], max_dim=max_dim,dataset_name=dataset_name)

In [30]:
optimizer = 'adam'
dtc.initialize()
dtc.model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Neighbour1_input (InputLayer)  [(None, None, 108)]  0           []                               
                                                                                                  
 Neighbour2_input (InputLayer)  [(None, None, 108)]  0           []                               
                                                                                                  
 input (InputLayer)             [(None, None, 108)]  0           []                               
                                                                                                  
 lstm_Neighbour1_256 (LSTM)     (None, None, 256)    373760      ['Neighbour1_input[0][0]']       
                                                                                              

In [31]:
dtc.compile(gamma=1.0, optimizer=optimizer, initial_heatmap_loss_weight=0.1,
                final_heatmap_loss_weight=0.9)

In [32]:
dtc.pretrain(X=reshaped_array, optimizer=optimizer, epochs=50, batch_size=10,save_dir="/kaggle/working/")

Pretraining...
Epoch 1/50
48/48 [==============================] - 9s 187ms/step - loss: 0.6233
Epoch 2/50
48/48 [==============================] - 9s 188ms/step - loss: 0.4904
Epoch 3/50
48/48 [==============================] - 9s 190ms/step - loss: 0.4706
Epoch 4/50
48/48 [==============================] - 9s 190ms/step - loss: 0.4562
Epoch 5/50
48/48 [==============================] - 9s 186ms/step - loss: 0.4453
Epoch 6/50
48/48 [==============================] - 9s 186ms/step - loss: 0.4315
Epoch 7/50
48/48 [==============================] - 9s 188ms/step - loss: 0.4218
Epoch 8/50
48/48 [==============================] - 9s 189ms/step - loss: 0.4107
Epoch 9/50
48/48 [==============================] - 9s 186ms/step - loss: 0.4032
Epoch 10/50
48/48 [==============================] - 9s 190ms/step - loss: 0.3767
Epoch 11/50
48/48 [==============================] - 9s 192ms/step - loss: 0.3555
Epoch 12/50
48/48 [==============================] - 9s 194ms/step - loss: 0.3481
Epoch 13/5

In [33]:
dtc.load_ae_neighbour_weights()

In [34]:
# Initialize clusters
dtc.init_cluster_weights(reshaped_array)

Initializing cluster...
15/15 [==============================] - 1s 38ms/step
Done!


In [35]:
# Fit model
dtc.fit(reshaped_array, y_train=None, X_val=None, y_val=None)

Training for 10 epochs.
Evaluating every 10 and saving model every 10 epochs.
15/15 [==============================] - 2s 108ms/step
Number of reliable samples: 474
Number of unreliable samples: 0
reconstruction_loss:  tf.Tensor(335.2123, shape=(), dtype=float32)
clustering_loss:  tf.Tensor(0.25831142, shape=(), dtype=float32)
15/15 [==============================] - 2s 109ms/step
Epoch: 1/10
Total Loss:  tf.Tensor(335.4706, shape=(), dtype=float32)
15/15 [==============================] - 2s 144ms/step
Number of reliable samples: 474
Number of unreliable samples: 0
reconstruction_loss:  tf.Tensor(325.27667, shape=(), dtype=float32)
clustering_loss:  tf.Tensor(0.2517398, shape=(), dtype=float32)
15/15 [==============================] - 2s 109ms/step
Epoch: 2/10
Total Loss:  tf.Tensor(325.5284, shape=(), dtype=float32)
15/15 [==============================] - 2s 148ms/step
Number of reliable samples: 474
Number of unreliable samples: 0
reconstruction_loss:  tf.Tensor(316.2582, shape=(),

In [36]:
# Evaluate
print('Performance (TRAIN)')
results = {}
q = dtc.model.predict([reshaped_array, reshaped_array, reshaped_array])[1]
p = dtc.target_distribution(q)
p_pred = p.argmax(axis=1)

Performance (TRAIN)
15/15 [==============================] - 2s 152ms/step


In [37]:
np.save(dataset_name+'_Labels.npy', p_pred) # save

In [38]:
p_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [39]:
np.unique(p_pred, return_counts=True)

(array([0, 1]), array([180, 294]))